In [1]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB, ComplementNB 
import nltk
import string
from datasets import load_dataset
import re
from nltk.corpus import stopwords
from nltk import pos_tag
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import mlflow
from sklearnex import patch_sklearn
from warnings import filterwarnings
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


In [2]:
filterwarnings("ignore")

In [3]:
ag_news_dataset = load_dataset("ag_news")
stop_words = set(stopwords.words("english"))

In [4]:
text_preprocess_types = [None, 'стемминг', 'лемматизация']
#text_preprocess_types = ['лемматизация']

#words_classes = ['N', 'NJ', 'NJV', 'ALL']
words_classes = ['ALL']

#frequency_filtration_types = [None, 'low', 'high', 'both']
frequency_filtration_types = [None, 'low']

n_components = [150, 250, 375, 500, 750]

#hidden_layer_size = [375, 500, 750]

In [5]:
def base_ag_news_preprocess(text):
    tokens = text.lower()

    # Удаление спец слов
    special_words = ['reuters', 'afp', 'ap', 'usatoday.com', 'forbes.com', 'target=/stocks/quickinfo/fullquote"' ]
    for word in special_words:
        tokens = tokens.replace(word, '')
    
    pattern = r'[&lt][^<>]*&gt'
    tokens = re.sub(pattern, '', tokens)
    
    # Удаление пунктуации и цифр
    #tokens = ''.join(i if i not in set(string.punctuation) - set('-') | set(string.digits) else ' ' for i in tokens)
    tokens = ''.join(i if i not in set(string.punctuation)  | set(string.digits) else ' ' for i in tokens)
    
    # Токенизация
    tokens = nltk.word_tokenize(tokens)
    
    # Удаление стоп слов
    #stop_wordsL = stop_words - {'no','not'}
    stop_wordsL = stop_words
    tokens = [word for word in tokens if (word not in stop_wordsL and word != '-')]
    return tokens

In [6]:
def different_ag_news_preprocess(tokens, preprocess_type, words_class):
    
    # Обработка слов
    if preprocess_type == 'лемматизация':
        lemmatizer = nltk.WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
    elif preprocess_type == 'стемминг':
        stemmer = nltk.PorterStemmer()
        tokens = [stemmer.stem(token) for token in tokens]
    
    # Обработка частей речи
    tokens = pos_tag(tokens)
    if words_class == 'N':
        tokens = [word for word, tag in tokens if tag.startswith('N')]
    elif words_class == 'NJ':
        tokens = [word for word, tag in tokens if tag.startswith('N') or tag.startswith('J')]
    elif words_class == 'NJV':
        tokens = [word for word, tag in tokens if tag.startswith('N') or tag.startswith('J') or tag.startswith('V')]
    elif words_class == 'ALL':
        tokens = [word for word, _ in tokens]
    
    return tokens  

In [7]:
def frequency_filtration(words_dictionary, frequency_filtration_type):
    if frequency_filtration_type == 'low':
        return dict([(key, value) for key, value in words_dictionary.items() if value >= 10 ])
    elif frequency_filtration_type == 'high':
        return dict([(key, value) for key, value in words_dictionary.items() if value <= 3000])
    elif frequency_filtration_type == 'both':
        return dict([(key, value) for key, value in words_dictionary.items() if 10 <= value <= 3000])
    else:
        return words_dictionary

In [8]:
def dummy(doc):
    return doc

In [9]:
def final_ag_news_preprocess(dataset, model_type):
    # Подготовка данных
    x_train = dataset['train']['text']
    y_train = dataset['train']['label']
    
    x_test = dataset['test']['text']
    y_test = dataset['test']['label']
    
    # Базовая обработка
    for i, text in enumerate(x_train):
        x_train[i] = base_ag_news_preprocess(text)
        
    for i, text in enumerate(x_test):
        x_test[i] = base_ag_news_preprocess(text)
    
    index = 0
    # Вариативная обработка
    for preprocess_type in text_preprocess_types: 
        for words_class in words_classes:         
            words = {}
            xtr = x_train
            xte = x_test
            
            # Обработка текстов
            for i, tokens in enumerate(xtr):
                final_tokens = different_ag_news_preprocess(tokens, preprocess_type, words_class)
                xtr[i] = final_tokens
                
                # Заполнение словаря
                for token in final_tokens:
                    if token not in words:
                        words[token] = 1
                    else:
                        words[token] += 1
            
            xte = [different_ag_news_preprocess(tokens, preprocess_type, words_class) for tokens in xte]
                    
            # Фильтрация по частоте
            for frequency_filtration_type in frequency_filtration_types: 
                filtered_words = frequency_filtration(words, frequency_filtration_type)
                token_length = len(filtered_words)
                
                # Векторизация слов
                word_list = sorted(filtered_words.keys())
                # Присвоение словам индексов
                words_indexed = {}
                for idx, word in enumerate(word_list):
                    words_indexed[word] = idx
                
                # TF-IDF
                vectorizer_TFIDF = TfidfVectorizer(vocabulary=words_indexed, preprocessor=dummy, tokenizer=dummy, dtype=np.float32)
                x_train_TFIDF = vectorizer_TFIDF.fit_transform(xtr)
                x_test_TFIDF = vectorizer_TFIDF.transform(xte)
                
                #LSI
                #for n_num in n_components:
                    #SVD_TFIDF = TruncatedSVD(n_components=n_num)
                    
                    #SVD_TFIDF.fit(x_train_TFIDF)
                    
                    #x_train_TFIDF_SVD = SVD_TFIDF.transform(x_train_TFIDF)
                    #x_test_TFIDF_SVD = SVD_TFIDF.transform(x_test_TFIDF)

                    # Построение классификаторов
                if frequency_filtration_type is not None:
                    clf_G = GaussianNB()
                    
                    # Обучение классификаторов
                    clf_G.fit(x_train_TFIDF.toarray(), y_train)
             
                    # Тестирование
                    prediction_G = clf_G.predict(x_test_TFIDF.toarray())
                        
                    # Метрики
                    score_G = f1_score(y_test, prediction_G, average='macro')
                            
                    mlflow.start_run(run_name=f'Gaussian_TFIDF_{preprocess_type}_{words_class}_{frequency_filtration_type}')
                    mlflow.log_param('model', clf_G.__class__.__name__)
                    mlflow.log_param('preprocess_type', preprocess_type)
                    mlflow.log_param('words_class', words_class)
                    mlflow.log_param('frequency_filtration_type', frequency_filtration_type)
                    mlflow.log_param('token_length', token_length)
                    #mlflow.log_param('n_components', n_num)
                    mlflow.log_param('word vectorizer', 'TFIDF')
                    mlflow.log_metric('macro_score', score_G)
                    mlflow.end_run()
                
                clf_B = BernoulliNB()
                clf_B.fit(x_train_TFIDF, y_train)
                prediction_B = clf_B.predict(x_test_TFIDF)
                score_B = f1_score(y_test, prediction_B, average='macro')
                
                mlflow.start_run(run_name=f'Bernoulli_TFIDF_{preprocess_type}_{words_class}_{frequency_filtration_type}')
                mlflow.log_param('model', clf_B.__class__.__name__)
                mlflow.log_param('preprocess_type', preprocess_type)
                mlflow.log_param('words_class', words_class)
                mlflow.log_param('frequency_filtration_type', frequency_filtration_type)
                mlflow.log_param('token_length', token_length)
                #mlflow.log_param('n_components', n_num)
                mlflow.log_param('word vectorizer', 'TFIDF')
                mlflow.log_metric('macro_score', score_B)
                mlflow.end_run()
                    
                index += 1
                print(f'Итерация {index}')

In [99]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(experiment_name="agNews_NB")

2025/04/08 17:47:33 INFO mlflow.tracking.fluent: Experiment with name 'agNews_NB' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/729937831637931560', creation_time=1744123653173, experiment_id='729937831637931560', last_update_time=1744123653173, lifecycle_stage='active', name='agNews_NB', tags={}>

In [138]:
final_ag_news_preprocess(ag_news_dataset, 'MLP')

🏃 View run Bernoulli_TFIDF_None_ALL_None at: http://127.0.0.1:5000/#/experiments/729937831637931560/runs/1ff0c2286500450b950740b5d881cb67
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/729937831637931560
Итерация 1
🏃 View run Gaussian_TFIDF_None_ALL_low at: http://127.0.0.1:5000/#/experiments/729937831637931560/runs/8e138d84148f4e9594f90ea7bd91e151
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/729937831637931560
🏃 View run Bernoulli_TFIDF_None_ALL_low at: http://127.0.0.1:5000/#/experiments/729937831637931560/runs/363c7deb881b43e9bbe0f2be76a09839
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/729937831637931560
Итерация 2
🏃 View run Bernoulli_TFIDF_стемминг_ALL_None at: http://127.0.0.1:5000/#/experiments/729937831637931560/runs/cf8c1d19761949a487c76d650d078ba2
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/729937831637931560
Итерация 3
🏃 View run Gaussian_TFIDF_стемминг_ALL_low at: http://127.0.0.1:5000/#/experiments/729937831637931560/runs

Ручные тесты


In [27]:
dataset = ag_news_dataset
preprocess_type = None
words_class = 'ALL'
frequency_filtration_type = None

# Подготовка данных
x_train = dataset['train']['text']
y_train = dataset['train']['label']
    
x_test = dataset['test']['text']
y_test = dataset['test']['label']
    
# Базовая обработка
for i, text in enumerate(x_train):
    x_train[i] = base_ag_news_preprocess(text)
        
for i, text in enumerate(x_test):
    x_test[i] = base_ag_news_preprocess(text)
    
words = {}
xtr = x_train
xte = x_test
            
# Обработка текстов
for i, tokens in enumerate(xtr):
    final_tokens = different_ag_news_preprocess(tokens, preprocess_type, words_class)
    xtr[i] = final_tokens
                
    # Заполнение словаря
    for token in final_tokens:
        if token not in words:
            words[token] = 1
        else:
            words[token] += 1
            
xte = [different_ag_news_preprocess(tokens, preprocess_type, words_class) for tokens in xte]
                    
# Фильтрация по частоте
filtered_words = frequency_filtration(words, frequency_filtration_type)
token_length = len(filtered_words)
                
# Векторизация слов
word_list = sorted(filtered_words.keys())
# Присвоение словам индексов
words_indexed = {}
for idx, word in enumerate(word_list):
    words_indexed[word] = idx
    
# TF-IDF
vectorizer_TFIDF = TfidfVectorizer(vocabulary=words_indexed,max_features=5000, preprocessor=dummy, tokenizer=dummy, dtype=np.float32)
x_train_TFIDF = vectorizer_TFIDF.fit_transform(xtr)
x_test_TFIDF = vectorizer_TFIDF.transform(xte)



In [28]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

In [30]:
f1_macro_scorer = make_scorer(f1_score, average='macro')

In [36]:
clf = MultinomialNB()

In [37]:
clf = clf.fit(x_train_TFIDF, y_train)

In [38]:
predictions = clf.predict(x_test_TFIDF.toarray())
macro_score = f1_score(y_test, predictions, average='macro')

In [39]:
print(macro_score)

0.8984204774199408


In [40]:
clf = ComplementNB()

In [41]:
clf = clf.fit(x_train_TFIDF, y_train)

In [42]:
predictions = clf.predict(x_test_TFIDF.toarray())
macro_score = f1_score(y_test, predictions, average='macro')

In [43]:
print(macro_score)

0.9011009276712132


In [29]:
parameters = {
    'alpha':[0.01, 0.1, 0.5, 1.0]
}

In [45]:
# Инициализация RandomizedSearchCV
search = GridSearchCV(
    ComplementNB(),
    param_grid=parameters,
    scoring=f1_macro_scorer,          # Метрика
    n_jobs=-1,                  # Использовать все ядра
)

In [46]:
search.fit(x_train_TFIDF, y_train)

GridSearchCV(estimator=ComplementNB(), n_jobs=-1,
             param_grid={'alpha': [0.01, 0.1, 0.5, 1.0]},
             scoring=make_scorer(f1_score, response_method='predict', average=macro))

In [47]:
print("Лучшие параметры:", search.best_params_)
print()
print("Лучший F1 macro score:", search.best_score_)


Лучшие параметры: {'alpha': 0.5}

Лучший F1 macro score: 0.8934474245948845
